In [1]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency, gdf_data

In [2]:
places = ["Kreuzberg", "Schöneberg"]
places_gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = places_gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]
places_gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((13.36823 52.49334, 13.36917 52.49315...",52.509382,52.482792,13.452930,13.368229,307579131,relation,55765,52.497644,13.411914,"Kreuzberg, Friedrichshain-Kreuzberg, Berlin, G...",boundary,administrative,0.628946
1,"POLYGON ((13.33629 52.46745, 13.33742 52.46695...",52.504935,52.454638,13.376562,13.336290,307581282,relation,55751,52.482157,13.355190,"Schöneberg, Tempelhof-Schöneberg, Berlin, Germany",boundary,administrative,0.613688


In [ ]:
places_gdf[["projected_area", "area_unit"]] = places_gdf.apply(gdf_data, axis=1)

In [5]:
import json
keys = []
nodes = []
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    keys = list(count_tag_frequency(nodes).keys())
keys

['addr',
 'natural',
 'leaf_type',
 'leaf_cycle',
 'highway',
 'amenity',
 'name',
 'barrier',
 'crossing',
 'tactile_paving',
 'wheelchair',
 'source',
 'entrance',
 'operator',
 'railway',
 'opening_hours',
 'denotation',
 'check_date',
 'contact',
 'website',
 'access',
 'traffic_signals',
 'bicycle_parking',
 'kerb',
 'ref',
 'shop',
 'capacity',
 'covered',
 'backrest',
 'brand',
 'man_made',
 'level',
 'fee',
 'surveillance',
 'material',
 'payment',
 'lamp_type',
 'phone',
 'bollard',
 'camera',
 'button_operated',
 'cuisine',
 'genus',
 'fire_hydrant',
 'public_transport',
 'description',
 'outdoor_seating',
 'memorial',
 'light',
 'emergency',
 'healthcare',
 'lit',
 'support',
 'lamp_mount',
 'diet',
 'toilets',
 'start_date',
 'taxon',
 'pump',
 'tourism',
 'colour',
 'direction',
 'office',
 'bus',
 'bicycle',
 'image',
 'species',
 'indoor_seating',
 'diameter_crown',
 'seamark',
 'network',
 'foot',
 'waste',
 'bench',
 'note',
 'survey',
 'leisure',
 'seats',
 'shelter',

In [56]:
import utm
from pyproj import CRS
import geopandas as gpd
def gdf_data(gdf):
    """Get the area of a polygon
    This method is not directly callable by the LLM"""
    places_dict = {}
    for index, row in gdf.iterrows():
        print(index)
        utm_zone = utm.latlon_to_zone_number(gdf.loc[[index], "lat"].values[0], gdf.loc[[index], "lon"].values[0])
        south = gdf.loc[[index], "lat"].values[0] < 0
        crs = CRS.from_dict({"proj": "utm", "zone": utm_zone, "south": south})
        epsg_code = crs.to_authority()[1]
        unit = list({ai.unit_name for ai in crs.axis_info})[0]
        gdf_projected = gdf.loc[[index],:].to_crs(epsg_code)
        area = gdf_projected.area.values[0]
        places_dict[row["display_name"]] = {"area":area,
                                            "unit":unit}

    return places_dict

In [57]:
gdf_data(places_gdf)

0
1


{'Kreuzberg, Friedrichshain-Kreuzberg, Berlin, Germany': {'epsg': '32633',
  'area': 10422350.162337855,
  'unit': 'metre'},
 'Schöneberg, Tempelhof-Schöneberg, Berlin, Germany': {'epsg': '32633',
  'area': 10647009.479142455,
  'unit': 'metre'}}